# データインポート

In [3]:
import numpy as np
import pandas as pd
import os
import sys
from datetime import datetime, timedelta

sys.path.append("../")
from backend.elastic_manager.elastic_manager import ElasticManager

from elasticsearch import Elasticsearch, helpers

# ElasticManager.es = Elasticsearch(hosts='http://doc-elastic:9200', http_auth=("elastic", "P@ssw0rd"), timeout=50000)
# ElasticManager.es

## 概要確認

In [4]:
# 全ショットデータ
df_angle = pd.read_csv("20210810/0to1K/log0/log000_角度とR2107311654.csv", encoding="shift-jis")
df_angle.head(5)

,DATE,TIME,SHOT数,角度,R半径,板厚SHOT,板厚,板形状SHOT,ピークtoボトム,ピーク距離,ボトム距離,ピークボトム距離
0,2021/07/31,16:54:25,1,15.0283,71.8934,20,1.406,21,5.7352,87.900,194.625,108.975
1,2021/07/31,16:54:27,2,15.5637,74.3223,21,1.466,22,5.8597,73.575,194.625,123.150
2,2021/07/31,16:54:29,3,17.0261,74.7403,22,1.477,23,5.8780,88.875,194.250,108.075
3,2021/07/31,16:54:31,4,18.2962,73.1533,23,1.428,24,5.3836,80.025,194.625,116.325
4,2021/07/31,16:54:33,5,17.0462,71.8284,24,1.435,25,5.1383,87.150,194.625,109.800


In [39]:
# 1ショット分のデータ
df_stroke = pd.read_csv("20210810/0to1K/log1/log000_ストロークと歪210731165425.csv", encoding="shift-jis")
df_stroke.head(10)

,DATE,TIME,INTERVAL,SHOT数,歪量,ストローク
0,2021/07/31,16:54:25,0,9,-7,19.79
1,2021/07/31,16:54:25,99,9,-7,19.79
2,2021/07/31,16:54:25,102,9,-6,19.76
3,2021/07/31,16:54:25,98,9,-6,19.76
4,2021/07/31,16:54:25,107,9,-5,19.75
5,2021/07/31,16:54:25,93,9,-5,19.75
6,2021/07/31,16:54:25,104,9,-6,19.77
7,2021/07/31,16:54:25,96,9,-6,19.77
8,2021/07/31,16:54:25,105,9,4,19.77
9,2021/07/31,16:54:25,95,9,4,19.77


In [6]:
ElasticManager.delete_index("j-20210731165425-data")
ElasticManager.delete_index("j-20210731165425-meta")

ElasticManager.create_index("j-20210731165425-data")
ElasticManager.create_index("j-20210731165425-meta")

[2021/10/14 04:48:58.365] INFO - elastic_manager.py#delete_index:100: j-20210731165425-data deleted.
[2021/10/14 04:48:58.491] INFO - elastic_manager.py#delete_index:100: j-20210731165425-meta deleted.
[2021/10/14 04:48:58.858] INFO - elastic_manager.py#create_index:217: create index 'j-20210731165425-data' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'j-20210731165425-data'}
[2021/10/14 04:48:59.455] INFO - elastic_manager.py#create_index:217: create index 'j-20210731165425-meta' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'j-20210731165425-meta'}


True

## ショット単位データ

### 日時列追加

In [8]:
df_angle["timestamp"] = df_angle["DATE"] + ' ' + df_angle["TIME"]
df_angle["timestamp"] = df_angle["timestamp"].map(lambda x: datetime.strptime(x, "%Y/%m/%d %H:%M:%S") - timedelta(hours=9))
df_angle.drop(columns=["DATE", "TIME"], inplace=True)

In [9]:
dict_angle = df_angle.to_dict(orient="records")
dict_angle

[{'SHOT数': 1,
  '角度': 15.0283,
  'R半径': 71.8934,
  '板厚SHOT': 20,
  '板厚': 1.406,
  '板形状SHOT': 21,
  'ピークtoボトム': 5.7352,
  'ピーク距離': 87.9,
  'ボトム距離': 194.625,
  'ピークボトム距離': 108.975,
  'timestamp': Timestamp('2021-07-31 07:54:25')},
 {'SHOT数': 2,
  '角度': 15.5637,
  'R半径': 74.3223,
  '板厚SHOT': 21,
  '板厚': 1.466,
  '板形状SHOT': 22,
  'ピークtoボトム': 5.8597,
  'ピーク距離': 73.575,
  'ボトム距離': 194.625,
  'ピークボトム距離': 123.15,
  'timestamp': Timestamp('2021-07-31 07:54:27')},
 {'SHOT数': 3,
  '角度': 17.0261,
  'R半径': 74.7403,
  '板厚SHOT': 22,
  '板厚': 1.477,
  '板形状SHOT': 23,
  'ピークtoボトム': 5.878,
  'ピーク距離': 88.875,
  'ボトム距離': 194.25,
  'ピークボトム距離': 108.075,
  'timestamp': Timestamp('2021-07-31 07:54:29')},
 {'SHOT数': 4,
  '角度': 18.2962,
  'R半径': 73.1533,
  '板厚SHOT': 23,
  '板厚': 1.428,
  '板形状SHOT': 24,
  'ピークtoボトム': 5.3836,
  'ピーク距離': 80.025,
  'ボトム距離': 194.625,
  'ピークボトム距離': 116.325,
  'timestamp': Timestamp('2021-07-31 07:54:31')},
 {'SHOT数': 5,
  '角度': 17.0462,
  'R半径': 71.8284,
  '板厚SHOT': 24,
  '板厚': 1.435,
 

In [10]:
actions = (
    {"_index": "j-20210731165425-meta", "_source": x} for x in dict_angle
)
helpers.bulk(ElasticManager.es, actions, chunk_size=5000, stats_only=True, raise_on_error=False)

(1000, 0)

## ストローク（サンプル単位データ）

In [11]:
import glob

files = glob.glob("20210810/0to1K/log1/*")
files.sort()

df_stroke = pd.read_csv(files[0], encoding="shift-jis")

for file in files[1:]:
    df = pd.read_csv(file, encoding="shift-jis")
    df_stroke = pd.concat([df_stroke, df])

df_stroke.head(5)

,DATE,TIME,INTERVAL,SHOT数,歪量,ストローク
0,2021/07/31,16:54:25,0,9,-7,19.79
1,2021/07/31,16:54:25,99,9,-7,19.79
2,2021/07/31,16:54:25,102,9,-6,19.76
3,2021/07/31,16:54:25,98,9,-6,19.76
4,2021/07/31,16:54:25,107,9,-5,19.75


In [12]:
df_stroke["timestamp"] = df_stroke["DATE"] + ' ' + df_stroke["TIME"]
df_stroke["timestamp"] = df_stroke["timestamp"].map(lambda x: datetime.strptime(x, "%Y/%m/%d %H:%M:%S") - timedelta(hours=9))
df_stroke.drop(columns=["DATE", "TIME"], inplace=True)

In [13]:
df_stroke.head(100)

,INTERVAL,SHOT数,歪量,ストローク,timestamp
0,0,9,-7,19.79,2021-07-31 07:54:25
1,99,9,-7,19.79,2021-07-31 07:54:25
2,102,9,-6,19.76,2021-07-31 07:54:25
3,98,9,-6,19.76,2021-07-31 07:54:25
4,107,9,-5,19.75,2021-07-31 07:54:25
...,...,...,...,...,...
95,104,9,2,18.29,2021-07-31 07:54:25
96,96,9,2,18.29,2021-07-31 07:54:25
97,103,9,-9,18.14,2021-07-31 07:54:25
98,97,9,-9,18.14,2021-07-31 07:54:25


### ミリ秒以下の追加

In [14]:
def func(x):
    return x["timestamp"] + timedelta(microseconds=int(x["INTERVAL"]))

df_stroke["timestamp"] = df_stroke.apply(func, axis=1)

In [15]:
df_stroke

,INTERVAL,SHOT数,歪量,ストローク,timestamp
0,0,9,-7,19.79,2021-07-31 07:54:25.000000
1,99,9,-7,19.79,2021-07-31 07:54:25.000099
2,102,9,-6,19.76,2021-07-31 07:54:25.000102
3,98,9,-6,19.76,2021-07-31 07:54:25.000098
4,107,9,-5,19.75,2021-07-31 07:54:25.000107
...,...,...,...,...,...
2939,99,1008,19,20.03,2021-07-31 08:31:21.000099
2940,100,1008,19,20.03,2021-07-31 08:31:21.000100
2941,99,1008,9,20.03,2021-07-31 08:31:21.000099
2942,101,1008,9,20.01,2021-07-31 08:31:21.000101


In [16]:
dict_stroke = df_stroke.to_dict(orient="records")
# dict_stroke

In [17]:
actions = (
    {"_index": "j-20210731165425-data", "_source": x} for x in dict_stroke
)
helpers.bulk(ElasticManager.es, actions, chunk_size=5000, stats_only=True, raise_on_error=False)

(2942132, 0)

## 角度外れ値除去

In [18]:
df_angle[df_angle["角度"] < 0]

,SHOT数,角度,R半径,板厚SHOT,板厚,板形状SHOT,ピークtoボトム,ピーク距離,ボトム距離,ピークボトム距離,timestamp
914,915,-100000.0,73.1096,934,1.512,935,4.2556,69.6,194.25,124.65,2021-07-31 08:28:23


In [19]:
df_angle_without_outlire = df_angle.drop(914)

In [20]:
df_angle_without_outlire[df_angle_without_outlire["角度"] < 0]

,SHOT数,角度,R半径,板厚SHOT,板厚,板形状SHOT,ピークtoボトム,ピーク距離,ボトム距離,ピークボトム距離,timestamp


### 再インポート

In [21]:
ElasticManager.delete_index("j-20210731165425-meta")
ElasticManager.create_index("j-20210731165425-meta")

[2021/10/14 04:56:51.467] INFO - elastic_manager.py#delete_index:100: j-20210731165425-meta deleted.
[2021/10/14 04:56:51.859] INFO - elastic_manager.py#create_index:217: create index 'j-20210731165425-meta' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'j-20210731165425-meta'}


True

In [22]:
dict_angle_without_outlire = df_angle_without_outlire.to_dict(orient="records")

In [23]:
actions = (
    {"_index": "j-20210731165425-meta", "_source": x} for x in dict_angle_without_outlire
)
helpers.bulk(ElasticManager.es, actions, chunk_size=5000, stats_only=True, raise_on_error=False)

(999, 0)

## 列名を変更してpkl出力

In [59]:
dir = "20210810/machine-j-20210731165425"

if not os.path.exists(dir):
    os.makedirs(dir)

In [60]:
import glob
import shutil

files = glob.glob("20210810/0to1K/log1/*")
files.sort()

def concat_timestamp_and_interval(x):
    return x["timestamp"] + timedelta(microseconds=int(x["CUM_INTERVAL"]))

sequential_number = 0

for file in files:
    df_stroke = pd.read_csv(file, encoding="shift-jis")

    # 累積INTERVAL
    df_stroke["CUM_INTERVAL"] = df_stroke["INTERVAL"].cumsum()
    # timestamp列
    df_stroke["timestamp"] = df_stroke["DATE"] + ' ' + df_stroke["TIME"]
    df_stroke["timestamp"] = df_stroke["timestamp"].map(lambda x: datetime.strptime(x, "%Y/%m/%d %H:%M:%S") - timedelta(hours=9))
    df_stroke["timestamp"] = df_stroke.apply(concat_timestamp_and_interval, axis=1)
    df_stroke["timestamp"] = df_stroke["timestamp"].map(lambda x: x.timestamp())
    
    # 連番
    df_stroke["sequential_number"] = df_stroke.index + sequential_number
    sequential_number += df_stroke.index[-1]

    df_stroke.drop(columns=["DATE", "TIME", "INTERVAL", "SHOT数", "CUM_INTERVAL"], inplace=True)
    df_stroke.rename(columns={"歪量":"load01", "ストローク":"stroke_displacement"}, inplace=True)
    
    pkl_file_name = os.path.splitext(os.path.basename(file))[0] + ".pkl"
    pkl_file_path = os.path.join(dir, pkl_file_name)
    
    # print(df_stroke.head())
    df_stroke.to_pickle(pkl_file_path)

   load01  stroke_displacement     timestamp  sequential_number
0      -7                19.79  1.627718e+09                  0
1      -7                19.79  1.627718e+09                  1
2      -6                19.76  1.627718e+09                  2
3      -6                19.76  1.627718e+09                  3
4      -5                19.75  1.627718e+09                  4
   load01  stroke_displacement     timestamp  sequential_number
0       3                19.85  1.627718e+09               2944
1       3                19.85  1.627718e+09               2945
2      -6                19.86  1.627718e+09               2946
3      -6                19.86  1.627718e+09               2947
4     -16                19.85  1.627718e+09               2948
   load01  stroke_displacement     timestamp  sequential_number
0       5                19.77  1.627718e+09               5898
1       5                19.77  1.627718e+09               5899
2      -7                19.79  1.627718

   load01  stroke_displacement     timestamp  sequential_number
0      -4                19.88  1.627718e+09              64977
1      -4                19.88  1.627718e+09              64978
2      -9                19.86  1.627718e+09              64979
3      -9                19.86  1.627718e+09              64980
4       9                19.86  1.627718e+09              64981
   load01  stroke_displacement     timestamp  sequential_number
0       3                19.76  1.627718e+09              67932
1       3                19.76  1.627718e+09              67933
2      -8                19.76  1.627718e+09              67934
3      -8                19.76  1.627718e+09              67935
4      -8                19.76  1.627718e+09              67936
   load01  stroke_displacement     timestamp  sequential_number
0      -2                19.89  1.627718e+09              70875
1      -8                19.89  1.627718e+09              70876
2      -8                19.87  1.627718

   load01  stroke_displacement     timestamp  sequential_number
0     -15                19.82  1.627718e+09             129817
1     -15                19.82  1.627718e+09             129818
2       7                19.81  1.627718e+09             129819
3       7                19.81  1.627718e+09             129820
4      -1                19.82  1.627718e+09             129821
   load01  stroke_displacement     timestamp  sequential_number
0      -5                19.75  1.627718e+09             132752
1      -5                19.75  1.627718e+09             132753
2      -9                19.75  1.627718e+09             132754
3      -9                19.75  1.627718e+09             132755
4      -5                19.75  1.627718e+09             132756
   load01  stroke_displacement     timestamp  sequential_number
0       3                19.89  1.627718e+09             135688
1       3                19.89  1.627718e+09             135689
2      -6                19.87  1.627718

   load01  stroke_displacement     timestamp  sequential_number
0      -9                19.89  1.627718e+09             194577
1      -9                19.89  1.627718e+09             194578
2      -4                19.85  1.627718e+09             194579
3      -4                19.85  1.627718e+09             194580
4      -1                19.86  1.627718e+09             194581
   load01  stroke_displacement     timestamp  sequential_number
0      12                19.80  1.627718e+09             197523
1       1                19.80  1.627718e+09             197524
2       1                19.78  1.627718e+09             197525
3       1                19.78  1.627718e+09             197526
4      -8                19.79  1.627718e+09             197527
   load01  stroke_displacement     timestamp  sequential_number
0      -5                19.84  1.627718e+09             200468
1      -5                19.84  1.627718e+09             200469
2      -8                19.82  1.627718

   load01  stroke_displacement     timestamp  sequential_number
0      -7                19.82  1.627718e+09             262303
1       9                19.82  1.627718e+09             262304
2       9                19.76  1.627718e+09             262305
3       9                19.76  1.627718e+09             262306
4      -1                19.77  1.627718e+09             262307
   load01  stroke_displacement     timestamp  sequential_number
0      12                19.87  1.627718e+09             265248
1      12                19.87  1.627718e+09             265249
2       0                19.89  1.627718e+09             265250
3       0                19.89  1.627718e+09             265251
4      -4                19.87  1.627718e+09             265252
   load01  stroke_displacement     timestamp  sequential_number
0      -7                19.76  1.627718e+09             268191
1      -7                19.76  1.627718e+09             268192
2     -15                19.74  1.627718

...

   load01  stroke_displacement     timestamp  sequential_number
0      -3                19.81  1.627720e+09            2670687
1      -3                19.81  1.627720e+09            2670688
2      -7                19.81  1.627720e+09            2670689
3      -7                19.80  1.627720e+09            2670690
4      -7                19.80  1.627720e+09            2670691
   load01  stroke_displacement     timestamp  sequential_number
0     -10                19.84  1.627720e+09            2673622
1     -10                19.84  1.627720e+09            2673623
2     -11                19.85  1.627720e+09            2673624
3     -11                19.85  1.627720e+09            2673625
4     -10                19.86  1.627720e+09            2673626
   load01  stroke_displacement     timestamp  sequential_number
0     -11                19.87  1.627720e+09            2676556
1     -11                19.87  1.627720e+09            2676557
2      -6                19.86  1.627720

   load01  stroke_displacement     timestamp  sequential_number
0      -5                19.78  1.627720e+09            2735305
1      -8                19.78  1.627720e+09            2735306
2      -8                19.76  1.627720e+09            2735307
3      -2                19.76  1.627720e+09            2735308
4      -2                19.78  1.627720e+09            2735309
   load01  stroke_displacement     timestamp  sequential_number
0       3                19.88  1.627720e+09            2738240
1       3                19.88  1.627720e+09            2738241
2       1                19.89  1.627720e+09            2738242
3       1                19.89  1.627720e+09            2738243
4     -14                19.88  1.627720e+09            2738244
   load01  stroke_displacement     timestamp  sequential_number
0       0                19.83  1.627720e+09            2741183
1       0                19.83  1.627720e+09            2741184
2      -5                19.81  1.627720

   load01  stroke_displacement     timestamp  sequential_number
0       0                19.87  1.627720e+09            2799958
1       0                19.87  1.627720e+09            2799959
2     -15                19.84  1.627720e+09            2799960
3     -15                19.84  1.627720e+09            2799961
4       2                19.83  1.627720e+09            2799962
   load01  stroke_displacement     timestamp  sequential_number
0      -4                19.76  1.627720e+09            2802904
1      -4                19.76  1.627720e+09            2802905
2      -5                19.74  1.627720e+09            2802906
3      -5                19.74  1.627720e+09            2802907
4      -3                19.76  1.627720e+09            2802908
   load01  stroke_displacement     timestamp  sequential_number
0     -14                19.81  1.627720e+09            2805838
1     -14                19.81  1.627720e+09            2805839
2       1                19.79  1.627720

   load01  stroke_displacement     timestamp  sequential_number
0      -8                19.77  1.627720e+09            2867603
1      -8                19.77  1.627720e+09            2867604
2     -10                19.77  1.627720e+09            2867605
3     -10                19.77  1.627720e+09            2867606
4       0                19.78  1.627720e+09            2867607
   load01  stroke_displacement     timestamp  sequential_number
0       2                19.89  1.627720e+09            2870547
1       2                19.89  1.627720e+09            2870548
2      -5                19.89  1.627720e+09            2870549
3      -5                19.89  1.627720e+09            2870550
4      16                19.89  1.627720e+09            2870551
   load01  stroke_displacement     timestamp  sequential_number
0      25                19.88  1.627720e+09            2873493
1      25                19.88  1.627720e+09            2873494
2      -6                19.86  1.627720

   load01  stroke_displacement     timestamp  sequential_number
0     -11                19.78  1.627720e+09            2935245
1     -11                19.78  1.627720e+09            2935246
2      -6                19.78  1.627720e+09            2935247
3      -6                19.78  1.627720e+09            2935248
4     -12                19.80  1.627720e+09            2935249
   load01  stroke_displacement     timestamp  sequential_number
0     -14                19.85  1.627720e+09            2938189
1     -14                19.85  1.627720e+09            2938190
2     -15                19.85  1.627720e+09            2938191
3     -15                19.85  1.627720e+09            2938192
4       2                19.85  1.627720e+09            2938193


In [53]:
df_stroke["timestamp"].map(lambda x: datetime.fromtimestamp(x))

0      2021-07-31 08:31:21.000000
1      2021-07-31 08:31:21.000095
2      2021-07-31 08:31:21.000195
3      2021-07-31 08:31:21.000295
4      2021-07-31 08:31:21.000395
                  ...            
2939   2021-07-31 08:31:21.293895
2940   2021-07-31 08:31:21.293995
2941   2021-07-31 08:31:21.294094
2942   2021-07-31 08:31:21.294195
2943   2021-07-31 08:31:21.294294
Name: timestamp, Length: 2944, dtype: datetime64[ns]

In [61]:
df_stroke

,load01,stroke_displacement,timestamp,sequential_number
0,-14,19.85,1.627720e+09,2938189
1,-14,19.85,1.627720e+09,2938190
2,-15,19.85,1.627720e+09,2938191
3,-15,19.85,1.627720e+09,2938192
4,2,19.85,1.627720e+09,2938193
...,...,...,...,...
2939,19,20.03,1.627720e+09,2941128
2940,19,20.03,1.627720e+09,2941129
2941,9,20.03,1.627720e+09,2941130
2942,9,20.01,1.627720e+09,2941131
